# Ray start/stop

In [ ]:
ray.init(ignore_reinit_error=True)

In [ ]:
import ray
ray.shutdown()

# Imports

In [45]:
%load_ext autoreload
%autoreload 2

import datetime as dt
import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 70)
pd.set_option('display.max_columns', 99)
import pandas_bokeh
pandas_bokeh.output_file("data/bokeh_output.html")

import polygon_df as pdf
import polygon_ds as pds
import polygon_s3 as ps3
import bar_samples as bs
import bar_labels as bl
import bar_workflow as bw
import utils_filters as ft
from utils_pickle import pickle_dump, pickle_load

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'
# pd.set_option('precision', 2)
# .plot_bokeh(sizing_mode="scale_height")
# set sampling params

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Sample tick dates

In [76]:
# set sampling params
symbol = 'VTI'
start_date = '2020-12-01'
end_date = '2021-02-01'
date = '2020-11-19'

thresh = {
    # mad filter
    'mad_value_winlen': 11,
    'mad_k': 22,    
    'mad_deviation_winlen': 333, 
    # jma filter
    'jma_winlen': 7,
    'jma_power': 2,
    # bar thresholds
    'renko_return': 'jma_return',
#     'renko_size': 0.11,
    'renko_reveral_multiple': 2,
    'renko_range_frac': 22,
    'max_duration_td': dt.timedelta(minutes=33),
    # min thresholds
    'min_duration_td': dt.timedelta(seconds=11),
    'min_tick_count': 33,
    # label params
    'label_reward_ratios': list(np.arange(2, 11, 0.5)),
}

ray_on = True

if ray_on:
    import ray
    ray.init(ignore_reinit_error=True)


2021-02-01 23:18:38,465	INFO worker.py:666 -- Calling ray.init() again after it has already been called.


In [ ]:
# bar flow
bar_dates2 = bw.bar_dates_workflow(symbol, start_date, end_date, thresh, ray_on=ray_on)

# fill daily gaps
stacked_df2 = bw.fill_gaps_dates(bar_dates, fill_col='jma_wmean')

# stacked stats
stats_df2 = stacked_df_stats(stacked_df)

(pid=3799) VTI 2020-12-04 trying to get data from local file...
(pid=3797) VTI 2020-12-03 trying to get data from local file...
(pid=3798) VTI 2020-12-08 trying to get data from local file...
(pid=3800) VTI 2020-12-07 trying to get data from local file...
(pid=3799) VTI 2020-12-09 trying to get data from local file...
(pid=3797) VTI 2020-12-10 trying to get data from local file...
(pid=3798) VTI 2020-12-11 trying to get data from local file...
(pid=3800) VTI 2020-12-14 trying to get data from local file...
(pid=3797) Dropping label, only 21 trades;start at: 2020-12-10 15:59:59.900325888-05:00
(pid=3797) VTI 2020-12-15 trying to get data from local file...
(pid=3798) VTI 2020-12-16 trying to get data from local file...
(pid=3800) VTI 2020-12-17 trying to get data from local file...
(pid=3799) Dropping label, only 17 trades;start at: 2020-12-09 15:59:58.609393664-05:00
(pid=3799) VTI 2020-12-18 trying to get data from local file...
(pid=3797) VTI 2020-12-21 trying to get data from local 

In [74]:
stacked_df.jma_wmean.plot_bokeh(sizing_mode="scale_height")

Figure(id='1382', ...)

In [75]:
results = []

for date in bar_dates:
    out = (date['ft_ticks_df'].status.value_counts() / date['ft_ticks_df'].shape[0]).to_dict()
    out.update({'date': date['date']})
    results.append(out)

pd.DataFrame(results)

,clean_open_market,mad_outlier,irregular_condition,ts_delta,clean_after_hours,clean_pre_market,zero_volume,date
0,0.978774,0.008687,0.007036,0.002811,0.002083,0.000550,0.000059,2020-11-02
1,0.983759,0.005360,0.005541,0.002519,0.001975,0.000786,0.000060,2020-11-03
2,0.974916,0.011011,0.005646,0.003792,0.003006,0.001545,0.000084,2020-11-04
3,0.978476,0.007270,0.005460,0.004109,0.003161,0.001437,0.000086,2020-11-05
4,0.975134,0.009754,0.007041,0.004362,0.002816,0.000790,0.000103,2020-11-06
5,0.963384,0.014440,0.006405,0.007267,0.002828,0.005619,0.000056,2020-11-09
6,0.984663,0.006851,0.004791,0.002060,0.001119,0.000448,0.000067,2020-11-10
7,0.979872,0.007280,0.004839,0.003597,0.003083,0.001199,0.000128,2020-11-11
8,0.983066,0.007233,0.003515,0.003224,0.002440,0.000436,0.000087,2020-11-12
9,0.980471,0.004892,0.006300,0.003632,0.003928,0.000667,0.000111,2020-11-13


In [ ]:
# rw, outlier_ind = ft.random_walk_with_outliers(0, 1000, 0.01)
# plt.plot(np.arange(len(rw)), rw)
# plt.scatter(outlier_ind, rw[outlier_ind], c='r', label='outlier')
# plt.title('Random Walk with outliers')
# plt.xlabel('Time steps')
# plt.ylabel('Values')
# plt.legend();

from bokeh.plotting import figure, output_file, show, output_notebook
# output_notebook()

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

p1 = figure(title="Legend Example", tools=TOOLS)
p1.line(df.index, df.price)
p1.circle(df[df.mad_outlier==True].index, df[df.mad_outlier==True].price, legend_label="outlier", color="orange")

# output_file("legend.html", title="legend.py example")
show(p1, sizing_mode="scale_height")  # open a browser